<a href="https://colab.research.google.com/github/colinb2025/Ds2002_ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Worked with Ollie Mills (ohm8nbf)

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np
import requests
from requests.structures import CaseInsensitiveDict

In [ ]:
questions = [
    'Hi',
    'Hello',
    'Hey',
    'What is your name?',
    'How are you?',
    'Goodbye',
    'Bye',
    'Talk to you again',
    'I am done',
    'Done',
]

answers = [
    'Hi, how can I help',
    'Hi, how can I help',
    'Hi, how can I help',
    'I am just a chat bot',
    'I am a chat bot, I do not feel anything',
    'Bye! Talk to you soon!',
    'Bye! Talk to you soon!',
    'Bye! I will talk to you soon!',
    'Okay, talk to you soon!',
    'Okay, talk to you soon!',
]

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

vocab_size = len(tokenizer.word_index) + 1

In [ ]:
embedding_dim = 256
units = 1024

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            8192      ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            8192      ['input_2[0][0]']             
                                                                                              

In [ ]:
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1

model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=200)

Epoch 1/200
5/5 [==============================] - 8s 977ms/step - loss: 2.6770 - accuracy: 0.3300
Epoch 2/200
5/5 [==============================] - 4s 762ms/step - loss: 2.4863 - accuracy: 0.4400
Epoch 3/200
5/5 [==============================] - 4s 747ms/step - loss: 2.1170 - accuracy: 0.4300
Epoch 4/200
5/5 [==============================] - 4s 882ms/step - loss: 1.9602 - accuracy: 0.4600
Epoch 5/200
5/5 [==============================] - 4s 807ms/step - loss: 2.2681 - accuracy: 0.3900
Epoch 6/200
5/5 [==============================] - 4s 818ms/step - loss: 2.1723 - accuracy: 0.4600
Epoch 7/200
5/5 [==============================] - 5s 943ms/step - loss: 1.7445 - accuracy: 0.4800
Epoch 8/200
5/5 [==============================] - 4s 772ms/step - loss: 1.6634 - accuracy: 0.5200
Epoch 9/200
5/5 [==============================] - 4s 759ms/step - loss: 1.8798 - accuracy: 0.5200
Epoch 10/200
5/5 [==============================] - 4s 869ms/step - loss: 1.6257 - accuracy: 0.4700
Epoch 11/

In [ ]:
def get_coordinates(address):
    url = f'https://api.geoapify.com/v1/geocode/search?text={address}&apiKey=a5ed664f71174443976e423f0095e46a'
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    response = requests.get(url, headers=headers)
    data = response.json()
    latitude = data['features'][0]['geometry']['coordinates'][1]
    longitude = data['features'][0]['geometry']['coordinates'][0]
    return latitude, longitude

def get_weather_forecast(latitude, longitude):
    url = f"https://api.tomorrow.io/v4/weather/forecast?location={latitude},{longitude}&apikey=xtqbM3gXDavJgIfFAHyzCOHEJDaD33Or"
    response = requests.get(url)
    data = response.json()
    return data

def convert_to_fahrenheit(celsius_temp):
    return (celsius_temp * 9/5) + 32

def data_query():
    print("Bot: What is the address or city?")
    address = input("You: ")
    latitude, longitude = get_coordinates(address)
    weather_data = get_weather_forecast(latitude, longitude)

    temperature_celsius = weather_data['timelines']['minutely'][0]['values']['temperature']
    temperature_fahrenheit = convert_to_fahrenheit(temperature_celsius)

    print(f"Bot: The current temperature in {address} is {temperature_fahrenheit:.2f}°F.")

def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence], verbose=0).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

print("Start chatting with the bot! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit' or user_input.lower() == 'done' or user_input.lower() == 'i am done':
        break
    if "weather" in user_input.lower():
        response = data_query()
    else:
      response = chat_with_bot(user_input)
      print(f"Bot: {response}")

Start chatting with the bot! Type 'quit' to exit.
Bot: hi how can help
Bot: What is the address or city?
Bot: The current temperature in Charlottesville is 79.03°F.
Bot: bye talk to
